### AI Model - Retinanet

In this section of the notebook, it will investigate the performance of Retinanet when being trained against the public dataset "AgroPest-12". 

Since Retinanet does not take YOLO formats directly, the YOLO format requires pre-processing and converted into useable data.

In [2]:
import torch
from PIL import Image

def yolo_converter(path, width, height):
    annots = []
    labels = []
    with open(path) as f:
        for line in f.readlines():
            class_id, x, y, w, h = map(float, line.strip().split())
            x_min = (x - w/2) * width
            y_min = (y - h/2) * height
            x_max = (x + w/2) * width
            y_max = (y + h/2) * height
            annots.append([x_min, y_min, x_max, y_max])
            labels.append(int(class_id) + 1)

    if len(annots) == 0:
        annots = torch.zeros((0, 4), dtype=torch.float32)
        labels = torch.zeros((0,), dtype=torch.int64)
    else:
        annots = torch.tensor(annots, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
    return annots, labels

In [3]:
from torch.utils.data import Dataset
from torchvision import transforms

trans = transforms.Compose([
    transforms.ToTensor(),
])


class ImgDataset(Dataset):
    def __init__(self, path, label_path, trans=None):
        self.path = path
        self.label_path = label_path
        self.trans = trans

    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, index):
        img_path = self.path[index]
        label_path = self.label_path[index]
        
        img = Image.open(img_path).convert("RGB")
        wid, height = img.size
        try:
            annots, labels = yolo_converter(label_path, wid, height)
        except:
            print("Path Not found!")
            exit(1)

        if self.trans:
            img = self.trans(img)
        target = {"boxes": annots, "labels": labels}
        
        return img, target

Once the data is paths and labels are pre-processed correctly, the data can be allocated to training, validation and testing sets.

In [4]:
import torch
import torchvision
from torchvision.transforms import functional as F
from torchvision.models.detection import retinanet_resnet50_fpn, RetinaNet_ResNet50_FPN_Weights
from glob import glob
 
train_img = sorted(glob("../2/train/images/*.jpg"))
train_lab = sorted(glob("../2/train/labels/*.txt"))

train = ImgDataset(train_img, train_lab, trans=trans)

val_img = sorted(glob("../2/valid/images/*.jpg"))
val_lab = sorted(glob("../2/valid/labels/*.txt"))
val = ImgDataset(val_img, val_lab, trans=trans)

test_img = sorted(glob("../2/test/images/*.jpg"))
test_lab = sorted(glob("../2/test/labels/*.txt"))
test = ImgDataset(test_img, test_lab, trans=trans)


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print("Successfully Initialsed datas")


Successfully Initialsed datas


In [5]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(train, batch_size=2, shuffle=True, collate_fn=collate_fn, num_workers=4, pin_memory=True)
test_loader = DataLoader(test, batch_size=2, shuffle=True, collate_fn=collate_fn, num_workers=4, pin_memory=True)
val_loader = DataLoader(val, batch_size=2, shuffle=True, collate_fn=collate_fn, num_workers=4, pin_memory=True)

print("Successfully loaded the image data into data sets")

Successfully loaded the image data into data sets


Once the data is loaded into data loader, the AI model can begain training.

In [6]:
weight = RetinaNet_ResNet50_FPN_Weights.DEFAULT
model = retinanet_resnet50_fpn(weights=weight)

classes = 12 + 1 # 1 more to accommodate the background
in_feature = model.head.classification_head.cls_logits.in_channels
num_anchors = model.head.classification_head.num_anchors

model.head.classification_head = torchvision.models.detection.retinanet.RetinaNetClassificationHead(
    in_channels=in_feature,
    num_anchors=num_anchors,
    num_classes=classes
)

print("Initialised Retinanet Model!")


Initialised Retinanet Model!


In [ ]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision

model.to(device)
metric = MeanAveragePrecision(iou_thresholds=[0.5, 0.95],
                              max_detection_thresholds=[1, 10, 100],
                              class_metrics=True)
metric.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

train_loss_arr = []
val_loss_arr = []
val_maps_arr = []

epoachs = 15
for epoach in range(epoachs):
    model.train()
    epoach_loss = 0
    for images, targets in train_loader:
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        
        epoach_loss += losses.item()
    
    train_loss = epoach_loss / len(train_loader)
    train_loss_arr.append(train_loss)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, targets in val_loader:
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            preds = model(images)
            metric.update(preds, targets)

    val_res = metric.compute()
    val_map = val_res['map']
    val_maps_arr.append(val_map.item())

    print(f"Epoch [{epoach+1}/{epoachs}]")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val mAP: {val_map:.4f}")

    torch.save(model.state_dict(), "retinanet_weights.pth")
    
    metric.reset()

torch.save(model.state_dict(), "retinanet_weights.pth")
print("Finished training and Exported model")

/home/james/.local/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)


Epoch [1/15]
Train Loss: 0.9335
Val mAP: 0.0651
Epoch [2/15]
Train Loss: 0.7586
Val mAP: 0.1497
Epoch [3/15]
Train Loss: 0.6383
Val mAP: 0.2436
Epoch [4/15]
Train Loss: 0.5582
Val mAP: 0.2671
Epoch [5/15]
Train Loss: 0.4894
Val mAP: 0.2856
Epoch [6/15]
Train Loss: 0.4391
Val mAP: 0.2879
Epoch [7/15]
Train Loss: 0.4061
Val mAP: 0.3109
Epoch [8/15]
Train Loss: 0.3730
Val mAP: 0.3150
Epoch [9/15]
Train Loss: 0.3442
Val mAP: 0.3211
Epoch [10/15]
Train Loss: 0.3228
Val mAP: 0.3107
Epoch [11/15]
Train Loss: 0.3070
Val mAP: 0.3091
Epoch [12/15]
Train Loss: 0.2972
Val mAP: 0.3262
Epoch [13/15]
Train Loss: 0.2861
Val mAP: 0.3216
Epoch [14/15]
Train Loss: 0.2769
Val mAP: 0.3038
Epoch [15/15]
Train Loss: 0.2664
Val mAP: 0.3123
Finished training and Exported model
